In [ ]:
#gitのクローン 
!git clone https://github.com/HirooHirose/ds_Hirose/

Cloning into 'ds_Hirose'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 7 (delta 0), reused 7 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (7/7), 20.56 KiB | 5.14 MiB/s, done.


In [2]:
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

df = pd.read_csv('./ds_Hirose/1605.csv')

# 日付をインデックスに変換し、時系列として扱う
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

df = df[['Open', 'High', 'Low', 'Close', 'Over']].astype(float)

# 自動的に頻度を推定して補完
df = df.asfreq(pd.infer_freq(df.index))

# 欠損値が発生した場合は線形補完（必要に応じて）
df = df.interpolate(method='linear')

# VARモデルの作成
model = VAR(df)

# 最適なラグ数の選択（最大15まで）
lag_order_results = model.select_order(maxlags=15)
optimal_lag = lag_order_results.aic

# モデルのフィッティング（AICで選ばれたラグ数を使用）
fitted_model = model.fit(optimal_lag)

# 終値の10日先まで予測
forecast_steps = 10
forecast = fitted_model.forecast(df.values[-optimal_lag:], steps=forecast_steps)

# 予測結果をDataFrameに変換
forecast_df = pd.DataFrame(forecast, columns=df.columns)
forecast_close = forecast_df['Close']

forecast_close

,Close
0,1678.204836
1,1677.590109
2,1672.352330
3,1677.078313
4,1673.409280
5,1670.700285
6,1668.733973
7,1668.249325
8,1667.546892
9,1667.015783
